In [1]:
!pip install git+https://github.com/Farama-Foundation/MAgent2

  Cloning https://github.com/Farama-Foundation/MAgent2 to /tmp/pip-req-build-2i38s54c
  Running command git clone --filter=blob:none --quiet https://github.com/Farama-Foundation/MAgent2 /tmp/pip-req-build-2i38s54c
  Resolved https://github.com/Farama-Foundation/MAgent2 to commit b2ddd49445368cf85d4d4e1edcddae2e28aa1406
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 96.4 MB/s eta 0:00:00
  Created wheel for magent2: filename=magent2-0.3.3-cp310-cp310-linux_x86_64.whl size=1696137 sha256=42cad2369f961d5a7edc538bc22523dd5b053e33db49084576b005f805d9be46
  Stored in directory: /tmp/pip-ephem-wheel-cache-150l3vdb/wheels/e4/8e/bf/51a30bc4038546e23b81c9fb513fe6a8fd916e5a9c5f4291d5
Successfully built magent2


In [2]:
!git clone https://github.com/giangbang/RL-final-project-AIT-3007.git

Cloning into 'RL-final-project-AIT-3007'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 56 (delta 12), reused 13 (delta 6), pack-reused 32 (from 1)
Receiving objects: 100% (56/56), 17.78 MiB | 20.20 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [3]:
import sys
sys.path.append('/kaggle/input/model-qnet')
from model_deep import *

In [4]:
import sys
sys.path.append('/kaggle/working/RL-final-project-AIT-3007')

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
from collections import deque, Counter
import os
from magent2.environments import battle_v4
import time
# from torch_model import QNetwork

class MyQNetwork(nn.Module):
    def __init__(self, observation_shape, action_shape):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(observation_shape[-1], 13, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(13, 13, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(13, 13, kernel_size=3),
            nn.ReLU(),
        )
        
        dummy_input = torch.randn(observation_shape).permute(2, 0, 1)
        dummy_output = self.cnn(dummy_input)
        flatten_dim = dummy_output.view(-1).shape[0]
        
        self.fc = nn.Sequential(
            nn.Linear(flatten_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            # nn.Linear(512, 256),
            # nn.ReLU(),
            # nn.Linear(256, 128),
            # nn.ReLU(),
            nn.Linear(128, action_shape)
        )

    def forward(self, x):
        assert len(x.shape) >= 3, "only support magent input observation"
        out = self.cnn(x)

        # print(out.shape)
        
        # out += self.skip_connection(x)

        # out = self.cnn2(out)
        
        if len(x.shape) == 3:
            batchsize = 1
            
        else:
            batchsize = x.shape[0]
            
        out = out.reshape(batchsize, -1)
        
        return self.fc(out)

class ReplayBuffer(Dataset):
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = deque(maxlen=capacity)
        
    def add(self, state, action, reward, next_state, done):
        experience = (state, action, reward, next_state, done)
        self.buffer.append(experience)

    def __len__(self):
        return len(self.buffer)

    def __getitem__(self, index):
        return self.buffer[index]

In [6]:
import torch
import torch.optim.lr_scheduler as lr_scheduler
class Trainer:
    def __init__(self, env, config_qnet=None, input_shape=None, action_shape=None, learning_rate=1e-3):
        self.env = env
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.q_network = MyQNetwork(input_shape, action_shape).to(self.device)
        # self.q_network = QNet(config_qnet, device=self.device)
        # self.q_network.load_state_dict(
        #     torch.load("/kaggle/input/blue-improved/blue_improved.pt", weights_only=True)
        # )
        # self.target_network = QNet(config_qnet, device=self.device)
        self.target_network = MyQNetwork(input_shape, action_shape).to(self.device)
        self.target_network.load_state_dict(self.q_network.state_dict())

        # self.red_pretrained_network = QNetwork(input_shape, action_shape).to(self.device)
        # self.red_pretrained_network.load_state_dict(
        #     torch.load("/kaggle/working/RL-final-project-AIT-3007/red.pt", weights_only=True)
        # )

        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.steplr = lr_scheduler.StepLR(optimizer=self.optimizer, step_size=1, gamma=0.9)
        self.replay_buffer = ReplayBuffer(capacity=16200 * 10)

        self.gamma = 0.9
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.96
        self.update_target_every = 2

    def select_action(self, observation, agent):
        if np.random.rand() <= self.epsilon:
            return self.env.action_space(agent).sample()

        observation = (
            torch.FloatTensor(observation).unsqueeze(0).to(self.device)
        )
        self.q_network.eval()
        with torch.inference_mode():
            q_values = self.q_network(observation)
            # print(q_values)
        return torch.argmax(q_values, dim=1).item()

    def pretrained_action(self, observation):
        observation = (
            torch.FloatTensor(observation).unsqueeze(0).to(self.device)
        )
        self.red_pretrained_network.eval()
        with torch.inference_mode():
            q_values = self.red_pretrained_network(observation)
        return torch.argmax(q_values, dim=1).item()

    def training(self, episodes=100, batch_size=2 ** 12):        
        for episode in range(episodes):
            self.env.reset()
            
            total_reward = 0
            reward_for_agent = {agent: 0 for agent in self.env.agents if agent.startswith('blue')}
            prev_observation = {}
            prev_action = {}
            self.env.reset()
            step = 0

            for idx, agent in enumerate(self.env.agent_iter()):
                step += 1
                observation, reward, termination, truncation, info = self.env.last()
                observation = np.transpose(observation, (2, 0, 1))
                
                agent_handle = agent.split('_')[0]
                
                if agent_handle == 'blue':
                    total_reward += reward
                    reward_for_agent[agent] += reward
                    
                if termination or truncation:
                    action = None
                else:
                    if agent_handle == 'blue':
                        action = self.select_action(observation, agent)
                    else:
                        action = self.env.action_space(agent).sample()
                        # action = self.pretrained_action(observation)

                if agent_handle == 'blue':
                    prev_observation[agent] = observation
                    prev_action[agent] = action
                
                self.env.step(action)
                
                if (idx + 1) % self.env.num_agents == 0:
                    break
                
            for agent in self.env.agent_iter():
                step += 1
                
                observation, reward, termination, truncation, info = self.env.last()
                observation = np.transpose(observation, (2, 0, 1))
                
                agent_handle = agent.split('_')[0]
                
                if agent_handle == 'blue':
                    total_reward += reward
                    reward_for_agent[agent] += reward
                    
                if termination or truncation:
                    action = None
                else:
                    if agent_handle == 'blue':
                        action = self.select_action(observation, agent)
                    else:
                        action = self.env.action_space(agent).sample()
                        # action = self.pretrained_action(observation)
    
                    if agent_handle == 'blue':
                        self.replay_buffer.add(
                            prev_observation[agent],
                            prev_action[agent],
                            reward,  
                            observation,
                            termination
                        )

                        prev_observation[agent] = observation
                        prev_action[agent] = action
    
                self.env.step(action)
            
            dataloader = DataLoader(self.replay_buffer, batch_size=batch_size, shuffle=True, drop_last=True)
            self.update_model(dataloader)
                
            if (episode + 1) % self.update_target_every == 0:
                self.target_network.load_state_dict(self.q_network.state_dict())
                self.steplr.step()
    
            max_reward = max(reward_for_agent.values())
            
            print(f"Episode {episode}, Epsilon: {self.epsilon:.2f}, Total Reward: {total_reward}, Steps: {step}, Max Reward: {max_reward}, lr: {self.steplr.get_last_lr()} ")
            self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)

    def update_model(self, dataloader):
        self.q_network.train()
        for states, actions, rewards, next_states, dones in dataloader:
            # print(states.shape)

            states = states.to(dtype=torch.float32, device=self.device)
            actions = actions.to(dtype=torch.long, device=self.device)
            rewards = rewards.to(dtype=torch.float32, device=self.device)
            next_states = next_states.to(dtype=torch.float32, device=self.device)
            dones = dones.to(dtype=torch.float32, device=self.device)

            current_q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
            with torch.inference_mode():
                next_q_values = self.target_network(next_states).max(1)[0]
            expected_q_values = rewards + (self.gamma * next_q_values * (1 - dones))

            loss = self.criterion(current_q_values, expected_q_values)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

In [7]:
env = battle_v4.env(map_size=45, render_mode=None)

config_model_qnet = {
    'layer_in':{
        'in_channels': env.observation_space("red_0").shape[-1],
        'out_channels': 64
    },
    'conv': {
        'num_layers': 1,
        'layer1':{
            'num_blocks': 2,
            'in_features': 64,
            'outs_features': [128,256,128],
        },
    },
    'mlp': {
        'num_layers': 1,
        'layer1':{
            'num_blocks': 2,
            'in_features': 128,
            'outs_features': [256,512,128],
        },
    },
    'layer_out':{
        'in_features': 128,
        'out_features': env.action_space("red_0").n
    }
}

trainer = Trainer(env, config_qnet=None, input_shape=env.observation_space("red_0").shape, action_shape=env.action_space("red_0").n)
trainer.training()

Episode 0, Epsilon: 1.00, Total Reward: -3230.5251185530797, Steps: 158820, Max Reward: -10.28000044170767, lr: [0.001] 
Episode 1, Epsilon: 0.96, Total Reward: -2970.395110585727, Steps: 156991, Max Reward: -6.375000230036676, lr: [0.0009000000000000001] 
Episode 2, Epsilon: 0.92, Total Reward: -3027.7801094865426, Steps: 158342, Max Reward: -11.105000398121774, lr: [0.0009000000000000001] 
Episode 3, Epsilon: 0.88, Total Reward: -2684.3500985521823, Steps: 154572, Max Reward: -9.10500031709671, lr: [0.0008100000000000001] 
Episode 4, Epsilon: 0.85, Total Reward: -2746.6701001953334, Steps: 155942, Max Reward: -25.900001152418554, lr: [0.0008100000000000001] 
Episode 5, Epsilon: 0.82, Total Reward: -1133.7450562650338, Steps: 62657, Max Reward: 1.9549992512911558, lr: [0.000729] 
Episode 6, Epsilon: 0.78, Total Reward: -2741.9000957002863, Steps: 156103, Max Reward: -24.700001201592386, lr: [0.000729] 
Episode 7, Epsilon: 0.75, Total Reward: -2325.70009253826, Steps: 118931, Max Rewar

In [8]:
os.makedirs("models", exist_ok=True)
torch.save(trainer.q_network.state_dict(), "models/blue_resnet_vs_random.pt")
print("Training complete. Model saved.")

Training complete. Model saved.


In [9]:
import torch.nn as nn
import torch


class finalQNetwork(nn.Module):
    def __init__(self, observation_shape, action_shape):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(observation_shape[-1], observation_shape[-1], 3),
            nn.ReLU(),
            nn.Conv2d(observation_shape[-1], observation_shape[-1], 3),
            nn.ReLU(),
        )
        dummy_input = torch.randn(observation_shape).permute(2, 0, 1)
        dummy_output = self.cnn(dummy_input)
        flatten_dim = dummy_output.view(-1).shape[0]
        self.network = nn.Sequential(
            nn.Linear(flatten_dim, 120),
            # nn.LayerNorm(120),
            nn.ReLU(),
            nn.Linear(120, 84),
            # nn.LayerNorm(84),
            nn.Tanh(),
        )
        self.last_layer = nn.Linear(84, action_shape)

    def forward(self, x):
        assert len(x.shape) >= 3, "only support magent input observation"
        x = self.cnn(x)
        if len(x.shape) == 3:
            batchsize = 1
        else:
            batchsize = x.shape[0]
        x = x.reshape(batchsize, -1)
        x = self.network(x)
        self.last_latent = x
        return self.last_layer(x)

class vQNetwork(nn.Module):
    def __init__(self, observation_shape, action_shape):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(observation_shape[-1], observation_shape[-1], 3),
            nn.ReLU(),
            nn.Conv2d(observation_shape[-1], observation_shape[-1], 3),
            nn.ReLU(),
        )
        dummy_input = torch.randn(observation_shape).permute(2, 0, 1)
        dummy_output = self.cnn(dummy_input)
        flatten_dim = dummy_output.view(-1).shape[0]
        self.network = nn.Sequential(
            nn.Linear(flatten_dim, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, action_shape),
        )

    def forward(self, x):
        assert len(x.shape) >= 3, "only support magent input observation"
        x = self.cnn(x)
        if len(x.shape) == 3:
            batchsize = 1
        else:
            batchsize = x.shape[0]
        x = x.reshape(batchsize, -1)
        return self.network(x)

In [10]:
# make video
import cv2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

env = battle_v4.env(map_size=45, render_mode="rgb_array", max_cycles=300)
vid_dir = "video"
os.makedirs(vid_dir, exist_ok=True)
fps = 35
frames = []
my_q_network = MyQNetwork(observation_shape=env.observation_space("red_0").shape, action_shape=env.action_space("red_0").n)

my_q_network.load_state_dict(
    torch.load("/kaggle/working/models/blue_resnet_vs_random.pt", weights_only=True, map_location="cpu")
)
my_q_network.to(device)
my_q_network.eval()

red_final_network = finalQNetwork(
    env.observation_space("red_0").shape, env.action_space("red_0").n
)
red_final_network.load_state_dict(
    torch.load("/kaggle/working/RL-final-project-AIT-3007/red_final.pt", weights_only=True, map_location="cpu")
)
red_final_network.to(device)
env.reset()
for agent in env.agent_iter():

    observation, reward, termination, truncation, info = env.last()
    observation = (
        torch.Tensor(observation).float().permute([2, 0, 1]).unsqueeze(0).to(device)
    )
    if termination or truncation:
        action = None  # this agent has died
    else:
        agent_handle = agent.split("_")[0]
        if agent_handle == "blue":
            with torch.inference_mode():
                q_values = my_q_network(observation)
            action = torch.argmax(q_values, dim=1).cpu().numpy()[0]
        else:
            # action = env.action_space(agent).sample()
            with torch.inference_mode():
                q_values = red_final_network(observation)
            action = torch.argmax(q_values, dim=1).item()
            
    env.step(action)

    if agent == "red_0":
        frames.append(env.render())

height, width, _ = frames[0].shape
out = cv2.VideoWriter(
    os.path.join(vid_dir, f"blue_vs_red_final.mp4"),
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (width, height),
)
for frame in frames:
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    out.write(frame_bgr)
out.release()
print("Done recording pretrained agents")

env.close()

Done recording pretrained agents


In [11]:
# from torch_model import QNetwork
try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x, *args, **kwargs: x  # Fallback: tqdm becomes a no-op
    
def eval():
    max_cycles = 300
    env = battle_v4.env(map_size=45, max_cycles=max_cycles)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    def random_policy(env, agent, obs):
        return env.action_space(agent).sample()

    # MyNetwork = QNet(config_model_qnet, device=device)
    MyNetwork = MyQNetwork(observation_shape=env.observation_space("red_0").shape, action_shape=env.action_space("red_0").n)
    
    MyNetwork.load_state_dict(
        torch.load("/kaggle/working/models/blue_resnet_vs_random.pt", weights_only=True, map_location="cpu")
    )
    MyNetwork.to(device)

    # red_final_network = vQNetwork(
    #     env.observation_space("red_0").shape, env.action_space("red_0").n
    # )
    # red_final_network.load_state_dict(
    #     torch.load("/kaggle/working/RL-final-project-AIT-3007/red_final.pt", weights_only=True, map_location="cpu")
    # ).to(device)

    red_final_network = finalQNetwork(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    red_final_network.load_state_dict(
        torch.load("/kaggle/working/RL-final-project-AIT-3007/red_final.pt", weights_only=True, map_location="cpu")
    )
    red_final_network = red_final_network.to(device)

    ###
    red_pretrained_network = vQNetwork(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    red_pretrained_network.load_state_dict(
        torch.load("/kaggle/working/RL-final-project-AIT-3007/red.pt", weights_only=True, map_location="cpu")
    )
    red_pretrained_network = red_pretrained_network.to(device)

    ###
    v1 = vQNetwork(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    v1.load_state_dict(
        torch.load("/kaggle/input/another-model/v1.pth", weights_only=True, map_location="cpu")
    )
    v1 = v1.to(device)

    ###
    v2 = vQNetwork(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    v2.load_state_dict(
        torch.load("/kaggle/input/another-model/v2.pth", weights_only=True, map_location="cpu")
    )
    v2 = v2.to(device)
    
    def pretrain_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        red_pretrained_network.eval()
        with torch.inference_mode():
            q_values = red_pretrained_network(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]
        
    def pretrain_policy_final(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        red_final_network.eval()
        with torch.inference_mode():
            q_values = red_final_network(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]
        
    def v1_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        v1.eval()
        with torch.inference_mode():
            q_values = v1(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]

    def v2_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        v2.eval()
        with torch.inference_mode():
            q_values = v2(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]

    def my_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        MyNetwork.eval()
        with torch.inference_mode():
            q_values = MyNetwork(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]

    # def policy()

    def run_eval(env, red_policy, blue_policy, n_episode: int = 100):
        red_win, blue_win = [], []
        red_tot_rw, blue_tot_rw = [], []
        n_agent_each_team = len(env.env.action_spaces) // 2

        for _ in tqdm(range(n_episode)):
            env.reset()
            n_kill = {"red": 0, "blue": 0}
            red_reward, blue_reward = 0, 0

            for agent in env.agent_iter():
                observation, reward, termination, truncation, info = env.last()
                agent_team = agent.split("_")[0]

                n_kill[agent_team] += (
                    reward > 4.5
                )  # This assumes default reward settups
                if agent_team == "red":
                    red_reward += reward
                else:
                    blue_reward += reward

                if termination or truncation:
                    action = None  # this agent has died
                else:
                    if agent_team == "red":
                        action = red_policy(env, agent, observation)
                    else:
                        action = blue_policy(env, agent, observation)

                env.step(action)

            who_wins = "red" if n_kill["red"] >= n_kill["blue"] + 5 else "draw"
            who_wins = "blue" if n_kill["red"] + 5 <= n_kill["blue"] else who_wins
            red_win.append(who_wins == "red")
            blue_win.append(who_wins == "blue")

            red_tot_rw.append(red_reward / n_agent_each_team)
            blue_tot_rw.append(blue_reward / n_agent_each_team)

        return {
            "winrate_red": np.mean(red_win),
            "winrate_blue": np.mean(blue_win),
            "average_rewards_red": np.mean(red_tot_rw),
            "average_rewards_blue": np.mean(blue_tot_rw),
        }

    print("=" * 20)
    print("Eval with final policy")
    print(
        run_eval(
            env=env, red_policy=pretrain_policy_final, blue_policy=my_policy, n_episode=30
        )
    )

    print("=" * 20)
    print("Eval with pretrain policy")
    print(
        run_eval(
            env=env, red_policy=pretrain_policy, blue_policy=my_policy, n_episode=30
        )
    )

    print("=" * 20)
    print("Eval with v1 policy")
    print(
        run_eval(
            env=env, red_policy=v1_policy, blue_policy=my_policy, n_episode=30
        )
    )
    print("=" * 20)

    print("Eval with v2 policy")
    print(
        run_eval(
            env=env, red_policy=v2_policy, blue_policy=my_policy, n_episode=30
        )
    )
    print("=" * 20)

if __name__ == "__main__":
    eval()

/opt/conda/lib/python3.10/site-packages/pettingzoo/utils/wrappers/base.py:72: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(


Eval with final policy


100%|██████████| 30/30 [02:02<00:00,  4.10s/it]


{'winrate_red': 0.0, 'winrate_blue': 1.0, 'average_rewards_red': 2.668094631581886, 'average_rewards_blue': 4.757100790606066}
Eval with pretrain policy


100%|██████████| 30/30 [01:26<00:00,  2.90s/it]


{'winrate_red': 0.0, 'winrate_blue': 1.0, 'average_rewards_red': 0.4954917650918933, 'average_rewards_blue': 4.881615192421671}
Eval with v1 policy


100%|██████████| 30/30 [01:46<00:00,  3.57s/it]


{'winrate_red': 0.3, 'winrate_blue': 0.5666666666666667, 'average_rewards_red': 4.166713969997004, 'average_rewards_blue': 4.5464691039914475}
Eval with v2 policy


100%|██████████| 30/30 [01:53<00:00,  3.78s/it]

{'winrate_red': 0.23333333333333334, 'winrate_blue': 0.4666666666666667, 'average_rewards_red': 4.381973228227239, 'average_rewards_blue': 4.449320954714476}
